# 导入packages

In [28]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import re
import os
import html
import matplotlib.pyplot as plt
import seaborn as sb

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec, Phrases
from sklearn.model_selection import train_test_split


In [29]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from scipy.sparse import hstack # 水平堆叠稀疏矩阵(按列)
from sklearn.feature_extraction.text import TfidfVectorizer #把原始文本转化为tf-idf的特征矩阵，文本相似度计算
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV


xgb = XGBClassifier()
lr = LogisticRegression()
r = RandomForestClassifier()



# other_params = {'learning_rate': 0.1, 'n_estimators': 100, 'max_depth': 5, 'min_child_weight': 1, 'seed': 0,
#                     'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

# param_test1 = {
#  'max_depth':list(range(2,6,9)),
#  'min_child_weight':list(range(2,4,6))
# }

# xgb1 = XGBClassifier(**other_params)
# gs1 = GridSearchCV(xgb1,param_test1,cv = 5,scoring = 'roc_auc',n_jobs = -1,verbose=2)
# best_model1=gs1.fit(x_train.values, y_train.values.reshape(-1))
# print('最优参数：',best_model1.best_params_)
# print('最佳模型得分：',best_model1.best_score_)

# 导入数据：数据探索性分析

In [80]:
# 本地路径
# train=pd.read_csv("E:/dasein_py/Data Analysis/nlp 大作业/IMDB/labeledTrainData.tsv",'\t', error_bad_lines= False)
# test=pd.read_csv("E:/dasein_py/Data Analysis/nlp 大作业/IMDB/testData.tsv","\t", error_bad_lines=False)
# unlabeled = pd.read_csv("E:/dasein_py/Data Analysis/nlp 大作业/IMDB/unlabeledTrainData.tsv", "\t", error_bad_lines=False)
# 在线路径 ../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip
train=pd.read_csv("../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip",'\t', error_bad_lines= False)
test=pd.read_csv("../input/word2vec-nlp-tutorial/testData.tsv.zip","\t", error_bad_lines=False)
unlabeled = pd.read_csv("../input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip", "\t", error_bad_lines=False)

In [77]:
train.head()

In [81]:
test.head()

In [33]:
unlabeled

In [34]:
# 本地路径
# att=pd.read_csv("E:/dasein_py/Data Analysis/nlp 大作业/IMDB/imdb_master.csv", error_bad_lines=False)
# 在线路径
# att=pd.read_csv("../input/imdb-review-dataset/imdb_master.csv", error_bad_lines=False, encoding = 'utf-8', engine = 'python')
# att.head()

In [35]:
# import chardet
# with open("../input/imdb-review-dataset/imdb_master.csv", 'rb') as f:
#     result = chardet.detect(f.read())
#     print(result['encoding'])

In [36]:
# att=att[["review","label"]]
# att.columns=["review","sentiment"]
# att=att[att.sentiment != "unsup"]
# att=att.replace({"neg":0,"pos":1})
# att.head()

In [82]:
train.shape, test.shape, unlabeled.shape

In [38]:
# df=train.append(att, ignore_index=True)
df=train.append(test, ignore_index=True)
df.info()

In [39]:
df.head()

# 数据清洗

In [40]:
df.review=html.unescape(df.review)
# 去除网页标记

df.review=df.review.str.replace('http\S+|www.\S+', '', case=False).str.replace(r"\&\#[0-9]+\;","", regex=True) \
            .str.replace(r"[^\w\s]","").str.replace("\d+","").str.replace(r"[\s]+"," ",regex=True) \
            .str.replace("\n"," ").replace("\r","").str.replace("_"," ").str.lower()

In [41]:
def remove_emoji(text):
    '''使用正则表达式去除网页标记'''
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" 
                           u"\U0001F300-\U0001F5FF" 
                           u"\U0001F680-\U0001F6FF"  
                           u"\U0001F1E0-\U0001F1FF"  
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
df['review']=df['review'].apply(lambda x: remove_emoji(x))

In [42]:
from unicodedata import normalize

remove_accent = lambda text: normalize("NFKD", text).encode("ascii", "ignore").decode("utf-8", "ignore") # 字符串标准化

df["review"] = df["review"].apply(remove_accent)

In [43]:
df.head()

In [44]:
df.sentiment.value_counts()

# 使用review建语料库，并分词

In [45]:
%%time
corpora = df["review"].values
tokenized = [word_tokenize(corpus) for corpus in corpora]

print(tokenized[2222])

In [46]:
%%time
tokenized = [list(filter(lambda x: len(x) > 1, document)) \
             for document in tokenized] 
# filter() 函数用于过滤序列，过滤掉不符合条件的元素，返回由符合条件元素组成的新列表。
print(tokenized[2222])

# Word Embedding for Feature Engineering

In [47]:
%%time
from gensim.models import word2vec
np.set_printoptions(suppress=True)

feature_size = 256
context_size = 5
min_word = 1

word_vec= word2vec.Word2Vec(tokenized, vector_size=feature_size, \
                            window=context_size, min_count=min_word, \
                            epochs=50, seed=42)

# 获得最终w2v的数据表

In [48]:
%%time
word_vec_unpack = [(word, idx) for word, idx in \
                   word_vec.wv.key_to_index.items()]

tokens, indexes = zip(*word_vec_unpack)

word_vec_df = pd.DataFrame(word_vec.wv.vectors[indexes, :], index=tokens)

word_vec_df.head()

In [49]:
%%time
tokenized_array = np.array(tokenized)

model_array = np.array([word_vec_df.loc[doc].mean(axis=0) for doc in tokenized_array])

In [50]:
model_df = pd.DataFrame(model_array)
model_df["sentiment"] = df["sentiment"]

model_df.head()

In [51]:
model_df.shape
model_df.isnull().sum()

In [52]:
# 手动train_test_split
df_train=model_df[:25000]
df_test=model_df[25000:]
y=df_train["sentiment"]

In [53]:
del df_train["sentiment"]
text=df_train

In [54]:
del df_test["sentiment"]
test=df_test

In [55]:
text.shape, y.shape, test.shape

In [56]:
X_train, X_test, y_train, y_test = train_test_split(text, y, test_size=0.2,random_state=42)
y_train.values

# 模型预测

In [57]:
%%time
r.fit(X_train,y_train.values) # 随机森林
pred=r.predict(X_test)
pred = pred.astype(int)
pred

In [58]:
print("classification_report")
print(classification_report(pred,y_test.values))

In [60]:
from sklearn.model_selection import GridSearchCV
other_params = {'n_estimators': 10, 
                'bootstrap': True, 
                }

param_test1 = {
 'n_estimators':list(range(10,20, 30)),
}

r1 = RandomForestClassifier(**other_params)
gs1 = GridSearchCV(r1,param_test1,cv = 5,scoring = 'roc_auc',n_jobs = -1,verbose=2)
best_model1=gs1.fit(X_train, y_train.values.reshape(-1))
print('最优参数：',best_model1.best_params_)
print('最佳模型得分：',best_model1.best_score_)

In [63]:
clr = RandomForestClassifier(n_estimators = 10)
r.fit(X_train,y_train.values) # 随机森林
pred=r.predict(X_test)
pred = pred.astype(int)
print("classification_report")
print(classification_report(pred,y_test.values))

In [86]:
sub=pd.DataFrame()
sub["id"]=test["id"]

pred=r.predict(df_test)
pred = pred.astype(int)

sub['sentiment'] = pd.DataFrame(pred)

In [87]:
sub.to_csv("submission_v1.csv", index = False)

In [91]:
%%time
LR = LogisticRegression(penalty = 'l2',
                       random_state = 42,
                       solver = 'sag',
                       max_iter = 100,
                       verbose = 0 )
LR.fit(X_train,y_train.values) # 逻辑斯蒂分类
pred=LR.predict(X_test)
pred = pred.astype(int)
print("classification_report")
print(classification_report(pred,y_test.values))

In [93]:
other_params = {'penalty': 'l2',
               'random_state': 42,
               'solver': 'sag',
               'max_iter': 100,
               'verbose': 0 
                }

param_test1 = {
 'max_iter':list(range(100, 300, 500)),
}

lr1 = LogisticRegression(**other_params)
gs1 = GridSearchCV(lr1,param_test1,cv = 5,scoring = 'roc_auc',n_jobs = -1,verbose=2)
best_model1=gs1.fit(X_train, y_train.values.reshape(-1))
print('最优参数：',best_model1.best_params_)
print('最佳模型得分：',best_model1.best_score_)

In [95]:
sub=pd.DataFrame()
sub["id"]=test["id"]

pred=LR.predict(df_test)
pred = pred.astype(int)

sub['sentiment'] = pd.DataFrame(pred)
sub.to_csv("submission_v2.csv", index = False)

In [96]:
# xgb = XGBClassifier()
other_params = {'learning_rate': 0.1, 'n_estimators': 100, 'max_depth': 5, 'min_child_weight': 1, 'seed': 0,
                    'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

param_test1 = {
 'max_depth':list(range(2,6,9)),
 'min_child_weight':list(range(2,4,6))
}

xgb = XGBClassifier(**other_params)
gs1 = GridSearchCV(xgb,param_test1,cv = 5,scoring = 'roc_auc',n_jobs = -1,verbose=2)
best_model1=gs1.fit(X_train, y_train.values.reshape(-1))
print('最优参数：',best_model1.best_params_)
print('最佳模型得分：',best_model1.best_score_)

In [100]:
other_params = {'learning_rate': 0.1, 'n_estimators': 500, 'max_depth': 2, 'min_child_weight': 2, 'seed': 0,
                    'subsample': 0.6, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}
xgb = XGBClassifier(**other_params)
xgb.fit(X_train,y_train.values) # 随机森林
pred = xgb.predict(X_test)
pred = pred.astype(int)
print("classification_report")
print(classification_report(pred,y_test.values))

In [101]:
sub=pd.DataFrame()
sub["id"]=test["id"]

pred=xgb.predict(df_test)
pred = pred.astype(int)

sub['sentiment'] = pd.DataFrame(pred)
sub.to_csv("submission_v3.csv", index = False)

* Random Forest Score: 0.82248
* Logistic Score: 0.87944
* XGBoost Score: 0.86352